In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data'
df = pd.read_csv(url,header=None)
df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

X = df.loc[:,2:].values
y = df.loc[:,1].values

le = LabelEncoder()
y = le.fit_transform(y)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,stratify=y,random_state=1)

# Learning Best Practices for Model Evaluation and Hyperparameter Tuning

## Chaining data preprocessing via Pipelines

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe_lr = make_pipeline(StandardScaler(),PCA(n_components=2),LogisticRegression())
pipe_lr.fit(X_train,y_train)
y_pred = pipe_lr.predict(X_test)
test_acc = pipe_lr.score(X_test,y_test)
print(f'Test Accuracy: {test_acc:.3f}')

## Acessing Model Performance using Stratified K-Fold Cross Validation to tune hyperparameters

In [ ]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10).split(X_train,y_train)

scores = []
for k, (train,test) in enumerate(kfold):

    pipe_lr.fit(X_train[train],y_train[train])
    score = pipe_lr.score(X_train[test],y_train[test])
    scores.append(score)
    print(f'Fold: {k+1:02d}, ',f'Class distr: {np.bincount(y_train[test])}, ',f'Acc.: {score:.3f}')
mean_acc = np.mean(scores)
std_acc = np.std(scores)
print(f'\nCV Accuracy: {mean_acc:.3f} +/- {std_acc:.3f}')


In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(estimator=pipe_lr,X=X_train,y=y_train,cv=10,n_jobs=1)
print(f'CV Accuracy scores: {scores}')
mean_acc = np.mean(scores)
std_acc = np.std(scores)
print(f'\nCV Accuracy: {mean_acc:.3f} +/- {std_acc:.3f}')

## Debugging Algos with Learning and Validation Curves

### Learning Curves

In [ ]:
from sklearn.model_selection import learning_curve

pipe_lr = make_pipeline(StandardScaler(),LogisticRegression(penalty='l2',max_iter=10000))

train_sizes,train_scores,test_scores = learning_curve(
    estimator=pipe_lr,
    X=X_train,
    y=y_train,
    train_sizes=np.linspace(.1,1.0,10),
    cv=10,
    n_jobs=1
)

train_mean = np.mean(train_scores,axis=1)
train_std = np.std(train_scores,axis=1)

test_mean = np.mean(test_scores,axis=1)
test_std = np.mean(test_scores,axis=1)

plt.plot(
    train_sizes,train_mean,
    color='blue',marker='o',
    markersize=5,label='Training Accuracy'
)
plt.fill_between(train_sizes,train_mean+train_std,train_mean-train_std,alpha=.15,color='blue')

plt.plot(
    train_sizes,test_mean,
    color='green',linestyle='--',
    marker='s',markersize=5,
    label='Validation Accuracy'
)
plt.fill_between(train_sizes,test_mean+test_std,test_mean-test_std,alpha=.15,color='green')

plt.grid()
plt.xlabel('Number of Training Exmaples')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.8,1.03])
plt.show()

### Validation Curves

In [ ]:
from sklearn.model_selection import validation_curve
param_range = [.001,.01,.1,1.0,10.0,100.0]
train_scores,test_scores = validation_curve(
    estimator=pipe_lr,
    X=X_train,
    y=y_train,
    param_name='logisticregression__C',
    param_range=param_range,
    cv=10
)

train_mean = np.mean(train_scores,axis=1)
train_std = np.std(train_scores,axis=1)

test_mean = np.mean(test_scores,axis=1)
test_std = np.std(test_scores,axis=1)

plt.plot(
    param_range,train_mean,
    color='blue',marker='o',
    markersize=5,label='Training Accuracy'
)
plt.fill_between(param_range,train_mean+train_std,train_mean-train_std,alpha=.15,color='blue')

plt.plot(
    param_range,test_mean,
    color='green',linestyle='--',
    marker='s',markersize=5,
    label='Validation Accuracy'
)
plt.fill_between(param_range,test_mean+test_std,test_mean-test_std,alpha=.15,color='green')

plt.grid()
plt.xscale('log')
plt.legend(loc='lower right')
plt.xlabel('Paramter C')
plt.ylabel('Accuracy')
plt.ylim([.8,1.0])
plt.show()

## Fine Tuning our Model via Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

pipe_svc = make_pipeline(
    StandardScaler(),
    SVC(random_state=1)
)

param_range = [.0001,.001,.01,.1,1.0,10.,100.,1000.]
param_grid = [
    {
        'svc__C':param_range,
        'svc__kernel':['linear']
    },
    {
        'svc__C':param_range,
        'svc__gamma':param_range,
        'svc__kernel':['rbf']
    }
]

gs = GridSearchCV(
    estimator=pipe_svc,
    param_grid=param_grid,
    scoring='accuracy',
    cv=10,
    refit=True,
    n_jobs=-1
)

gs = gs.fit(X_train,y_train)
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
clf = gs.best_estimator_
clf.fit(X_train,y_train)
print(f'Test Accuracy: {clf.score(X_test,y_test):.3f}')

## Fine Tuning Models via Randomized Search

In [ ]:
import scipy.stats
param_range = [.0001,.001,.01,.1,1.,10.,100.,1000.]

param_range = scipy.stats.loguniform(.0001,1000.)
np.random.seed(1)

param_range.rvs(10)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

pipe_svc = make_pipeline(
    StandardScaler(),
    SVC(random_state=1)
)

param_grid = [
    {
        'svc__C':param_range,
        'svc__kernel':['linear'],
    },
    {
        'svc__C':param_range,
        'svc__gamma':param_range,
        'svc__kernel':['rbf']
    }
]

rs = RandomizedSearchCV(
    estimator=pipe_svc,
    param_distributions=param_grid,
    scoring='accuracy',
    refit=True,
    n_iter=20,
    cv=10,
    random_state=1,
    n_jobs=-1
)

rs = rs.fit(X_train,y_train)
print(rs.best_score_)
print(rs.best_params_)

## Fine Tuning Models via Successive Halfing (Book example not working, try to figure this out later on)

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

hs = HalvingRandomSearchCV(
    pipe_svc,
    param_distributions=param_grid[1],
    n_candidates='exhaust',
    resource='n_samples',
    factor=1.5,
    random_state=1,
    n_jobs=-1
)

hs = hs.fit(X_train,y_train)
print(hs.best_score_)
print(hs.best_params_)
clf = hs.best_estimator_
print(f'Test Accuracy: {hs.score(X_test,y_test):.3f}')

## Algorithm Selection via Nested Cross-Validation

In [ ]:
param_range = [.0001,.001,.01,.1,1.,10.,100.,1000.]
param_grid = [
    {
        'svc__C':param_range,
        'svc__kernel':['linear'],
    },
    {
        'svc__C':param_range,
        'svc__gamma':param_range,
        'svc__kernel':['rbf']
    }
]
gs = GridSearchCV(
    estimator=pipe_svc,
    param_grid=param_grid,
    scoring='accuracy',
    cv=2
)
scores = cross_val_score(
    gs, X_train, y_train,
    scoring='accuracy',cv=5
)
print(f'CV Accuracy: {np.mean(scores):.3f} +/- {np.std(scores):.3f}')

In [ ]:
from sklearn.tree import DecisionTreeClassifier
gs = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=0),
    param_grid=[
        {'max_depth':[1,2,3,4,5,6,7,None]}
    ],
    scoring='accuracy',
    cv=2
)
scores = cross_val_score(
    gs,X_train,y_train,
    scoring='accuracy',cv=5
)
print(f'CV Accuracy: {np.mean(scores):.3f} +/- {np.std(scores):.3f}')

## Other Validation Metrics

### Confusion Matrices

In [ ]:
from sklearn.metrics import confusion_matrix
pipe_svc.fit(X_train,y_train)
y_pred = pipe_svc.predict(X_test)
confmat = confusion_matrix(y_true=y_test,y_pred=y_pred)


fig,ax = plt.subplots(figsize=(2.5,2.5))
ax.matshow(confmat,cmap=plt.cm.Blues,alpha=.3)
for i in range(confmat.shape[0]):
    for j in range(confmat.shape[1]):
        ax.text(x=j,y=i,s=confmat[i,j],va='center',ha='center')
ax.xaxis.set_ticks_position('bottom')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

### Precision and Recall Optimization

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score,  matthews_corrcoef

pre_val = precision_score(y_true=y_test,y_pred=y_pred)
print(f'Precision: {pre_val:.3f}')

rec_val = recall_score(y_true=y_test,y_pred=y_pred)
print(f'Recall: {rec_val:.3f}')

f1_val = f1_score(y_true=y_test,y_pred=y_pred)
print(f'F1: {f1_val:.3f}')

mcc_val = matthews_corrcoef(y_true=y_test,y_pred=y_pred)
print(f'MCC: {mcc_val:.3f}')

### Custom Scoring Solution to change the label we base our scoring off of

In [ ]:
from sklearn.metrics import make_scorer

c_gamma_range = [.01,.1,1.,10.]
param_grid = [
    {
        'svc__C':c_gamma_range,
        'svc__kernel':['linear']
    },
    {
        'svc__C':c_gamma_range,
        'svc__gamma':c_gamma_range,
        'svc__kernel':['rbf']
    }
]
scorer = make_scorer(f1_score,pos_label=0)
gs = GridSearchCV(
    estimator=pipe_svc,
    param_grid=param_grid,
    scoring=scorer,
    cv=10
)

gs = gs.fit(X_train,y_train)
print(gs.best_score_)
print(gs.best_params_)

## Reciever Operating Characteristic (ROC) and ROC Area Under Curve (ROC AUC)

In [ ]:
from sklearn.metrics import roc_curve,auc
from numpy import interp

pipe_lr = make_pipeline(
    StandardScaler(),
    PCA(n_components=2),
    LogisticRegression(
        penalty='l2',
        random_state=1,
        solver='lbfgs',
        C=100.
    )
)

X_train2 = X_train[:,[4,14]]
cv = list(StratifiedKFold(n_splits=3).split(X_train,y_train))

fig = plt.figure(figsize=(7,5))
mean_tpr = 0.
mean_fpr = np.linspace(0,1,100)
all_tpr = []

for i, (train,test) in enumerate(cv):
    probas = pipe_lr.fit(
        X_train2[train],
        y_train[train]
    ).predict_proba(X_train2[test])

    fpr,tpr,threshold = roc_curve(
        y_train[test],
        probas[:,1],
        pos_label=1
    )

    mean_tpr+=interp(mean_fpr,fpr,tpr)
    mean_tpr[0] = 0.
    roc_auc =auc(fpr,tpr)
    plt.plot(fpr,tpr,label=f'ROC fold {i+1} (area = {roc_auc:.2f})')

plt.plot(
    [0,1],
    [0,1],
    linestyle='--',
    color=(.6,.6,.6),
    label='Random Guessing (area=.5)',
)

mean_tpr /= len(cv)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr,mean_tpr)
plt.plot(
    mean_fpr,mean_tpr,'k--',label=f'Mean ROC (area={mean_auc:.2f})',
    lw=2
)

plt.plot(
    [0,0,1],
    [0,1,1],
    linestyle=':',
    color='black',
    label='Perfect Performance (area=1.0)'
)

plt.xlim([-.05,1.05])
plt.ylim([-.05,1.05])

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.legend(loc='lower right')
plt.show()

### Example of selecting Micro average when building our scores

In [ ]:
pre_scorer = make_scorer(
    score_func=precision_score,
    pos_label=1,
    greater_is_better=True,
    average='micro'
)

## Dealing with Class Imbalance

In [ ]:
X_imb = np.vstack((X[y==0],X[y==1][:40]))
y_imb = np.hstack((y[y==0],y[y==1][:40]))

#Prediction on imbalanced data by choosing the most prevelant data
y_pred = np.zeros(y_imb.shape[0])
np.mean(y_pred == y_imb) * 100

### Resampling and Upsampling the minority class

In [ ]:
from sklearn.utils import resample

X_upsampled,y_upsampled = resample(
    X_imb[y_imb==1],
    y_imb[y_imb==1],
    replace=True,
    n_samples=X_imb[y_imb==0].shape[0],
    random_state=123
)

print(f'Number of class 1 examples before: ',X_imb[y_imb==1].shape[0])
print(f'Number of class 1 exampels after: ',X_upsampled.shape[0])

In [ ]:
X_bal = np.vstack((X[y==0],X_upsampled))
y_bal = np.hstack((y[y==0],y_upsampled))

#Random guessing the majority class now has lower accuracy, which means our model can actually learn useful information
y_pred = np.zeros(y_bal.shape[0])
np.mean(y_pred == y_bal)*100